In [1]:
import numpy as np
import pandas as pd
import re
import scipy.stats as stats
import pylab
import gc

In [2]:
def linear_regression(X, y):
    X = pd.get_dummies(X, drop_first = True)._get_numeric_data().values
    y = y.values
    
    intercept_col = np.ones(X.shape[0]).reshape(X.shape[0],1)
    X = np.hstack((intercept_col, X))
    b = np.linalg.inv(X.T@X)@X.T@y
    return b

def rmse(slopes, X_test, y_test):
    X_test = pd.get_dummies(X_test, drop_first = True)._get_numeric_data().values
    intercept_col = np.ones(X_test.shape[0]).reshape(X_test.shape[0],1)
    X_test = np.hstack((intercept_col, X_test))
    
    y_val_pred = X_test @ b
    y_val = y_test.values
    return np.sqrt(((y_val - y_val_pred) ** 2).mean())
    
def train_test_split(frac, X, response = 'Volume Sold (Gallons)'):
    X_train = X.sample(frac = frac)
    X_test = X.drop(X_train.index, axis = 0)
    
    y_train = X_train[response]
    X_train = X_train.drop(['County','Area Name',response], axis = 1)
    
    y_test = X_test[response]
    X_test = X_test.drop(['County','Area Name',response], axis = 1)
    
    return X_train,X_test, y_train, y_test

def r2(b, X, y):
    X = pd.get_dummies(X, drop_first = True)._get_numeric_data().values
    y = y.values
    intercept_col = np.ones(X.shape[0]).reshape(X.shape[0],1)
    X = np.hstack((intercept_col, X))
    
    ss_tot = ((y - y.mean()) ** 2).sum()
    
    y_val_pred = X @ b
    ss_res = ((y - y_val_pred) ** 2).sum()
    return 1 - (ss_res/ss_tot)

def adj_r2(b, X, y):
    X = pd.get_dummies(X, drop_first = True)._get_numeric_data().values
    y = y.values
    intercept_col = np.ones(X.shape[0]).reshape(X.shape[0],1)
    X = np.hstack((intercept_col, X))
    
    ss_tot = ((y - y.mean()) ** 2).sum()
    
    y_val_pred = X @ b
    ss_res = ((y - y_val_pred) ** 2).sum()
    
    r2 = 1 - (ss_res/ss_tot)
    return 1 - (((1 - r2) * (X.shape[0] - 1)) / (X.shape[0] - X.shape[1] - 1))

def qqplot(b, X, y):
    X = pd.get_dummies(X, drop_first = True)._get_numeric_data().values
    y = y.values
    intercept_col = np.ones(X.shape[0]).reshape(X.shape[0],1)
    X = np.hstack((intercept_col, X))
    
    y_val_pred = X @ b

    stats.probplot(y - y_val_pred, dist="norm", plot=pylab)
    pylab.show()

def aic(b, X, y):
    X = pd.get_dummies(X, drop_first = True)._get_numeric_data().values
    y = y.values
    intercept_col = np.ones(X.shape[0]).reshape(X.shape[0],1)
    X = np.hstack((intercept_col, X))
    y_val_pred = X @ b
    
    ss_tot = ((y - y.mean()) ** 2).sum()
    ss_res = ((y - y_val_pred) ** 2).sum()
    sigmasq = ss_tot / X.shape[0]
    p = X.shape[1]
    
    return (ss_res + 2 *(p * sigmasq)) / ss_tot

def bic(b, X, y):
    X = pd.get_dummies(X, drop_first = True)._get_numeric_data().values
    y = y.values
    intercept_col = np.ones(X.shape[0]).reshape(X.shape[0],1)
    X = np.hstack((intercept_col, X))
    y_val_pred = X @ b
    
    ss_tot = ((y - y.mean()) ** 2).sum()
    ss_res = ((y - y_val_pred) ** 2).sum()
    sigmasq = ss_tot / X.shape[0]
    p = X.shape[1]
    n = X.shape[0]
    
    return (ss_res + (np.log(n) * p * sigmasq)) / ss_tot

In [ ]:
# LOAD IN MAIN DATASET
df = pd.read_csv("data/iowaliquor.csv")
# df_sample = pd.read_csv("data/iowa-sample.csv")

In [ ]:
# LOAD IN CENSUS DATASET
df_raw_county = pd.read_csv("data/co-est2019-alldata.csv", encoding = "ISO-8859-1")
df_raw_county["id"] = df_raw_county.index

In [ ]:
df_county_year = pd.wide_to_long(df_raw_county.drop(["SUMLEV","REGION","DIVISION","STATE","COUNTY","CENSUS2010POP","ESTIMATESBASE2010"],axis = 1),
                  ["POPESTIMATE","NPOCHG_","BIRTHS",
                  "DEATHS","NATURALINC","INTERNATIONALMIG",
                  "DOMESTICMIG","NETMIG","RESIDUAL",
                  "GQESTIMATESBASE","RBIRTH","RDEATH","RNATURALINC",
                  "RINTERNATIONALMIG","RDOMESTICMIG","RNETMIG","NPOPCHG_","GQESTIMATES"], i="id", j="year")

df_county_year = df_county_year[df_county_year["STNAME"] == "Iowa"]
df_county_year["CTYNAME"] = df_county_year["CTYNAME"].str.replace(" County", "").str.lower()
df_county_year = df_county_year.drop("NPOCHG_", axis = 1).dropna(axis = 1)
df_county_year = df_county_year[["CTYNAME","POPESTIMATE"]]

In [ ]:
df_county_year.head()

In [ ]:
# LOAD IN EMPLLOYMENT DATA
df_raw_employment = pd.read_csv("data/Iowa_Quarterly_Census_of_Employment_and_Wage_data__Statewide_and_County_.csv")
df_raw_employment.fillna(0)

In [ ]:
df_employement = pd.pivot_table(df_raw_employment,
               index = ["Year", "Quarter", "Area Name"],
               values = ["Average Emp", "Wages"],
               aggfunc = np.sum
              ).reset_index()

df_employement["Area Name"] = df_employement["Area Name"].str.lower()

In [ ]:
df_employement["Year"] = df_employement["Year"].astype('str')
df_employement["Quarter"] = df_employement["Quarter"].astype('str')

In [ ]:
# df_employement.head()

In [ ]:
df["County"] = df["County"].str.lower()

In [ ]:
df["Year"] = pd.DatetimeIndex(df['Date']).year
df["Quarter"] = pd.DatetimeIndex(df['Date']).quarter

In [ ]:
df["Year"] = df["Year"].astype('str')
df["Quarter"] = df["Quarter"].astype('str')
df["Pack"] = df["Pack"].astype('str')

In [ ]:
GROUP_VARIABLES = ['County','Year','Quarter']
NUMERIC_VARIABLES = ['Bottle Volume (ml)','State Bottle Cost', 'State Bottle Retail',
                     'Bottles Sold','Sale (Dollars)', 'Volume Sold (Liters)', 'Volume Sold (Gallons)']

df_agg =  pd.pivot_table(df, 
                        index = GROUP_VARIABLES,
                        values = NUMERIC_VARIABLES,
                        aggfunc = np.sum).reset_index().merge(df_employement, 'inner', left_on = ['Year','County', 'Quarter'], right_on = ['Year','Area Name', 'Quarter']).dropna()

In [ ]:
df_county_year = df_county_year.reset_index()

In [ ]:
df_agg["Year"] = df_agg["Year"].astype('int32')

In [ ]:
df_agg = df_agg.merge(df_county_year, 'inner', left_on = ['Year','County'], right_on = ['year','CTYNAME']).dropna()

In [ ]:
df_agg["Average Emp per Capita"] = df_agg["Average Emp"] / df_agg["POPESTIMATE"]
df_agg["Wages per Capita"] = df_agg["Wages"] / df_agg["POPESTIMATE"]
df_agg["Volume Sold (Gallons) per Capita"] = df_agg["Volume Sold (Gallons)"] / df_agg["POPESTIMATE"]
df_agg["Volume Sold (Liters) per Capita"] = df_agg["Volume Sold (Liters)"] / df_agg["POPESTIMATE"]

In [ ]:
# dollars per capita is skewed right
df_agg["sqrt(Sale (Dollars) per Capita)"] = np.sqrt(df_agg["Sale (Dollars)"] / df_agg["POPESTIMATE"])

In [ ]:
df_agg["Bottles Sold per Capita"] = df_agg["Bottles Sold"] / df_agg["POPESTIMATE"]
df_agg["Bottle Volume (ml) per Capita"] = df_agg["Bottle Volume (ml)"] / df_agg["POPESTIMATE"]

In [ ]:
df_agg.head()

In [ ]:
#---------------------------------
# 'Volume Sold (Liters) per Capita' AS RESPONSE VARIABLE
#---------------------------------

In [ ]:
X_train,X_test, y_train, y_test = train_test_split(.5, df_agg,"sqrt(Sale (Dollars) per Capita)")

VARS = ["Year","Quarter","Average Emp per Capita","Wages per Capita"] 
b = linear_regression(X_train[VARS], y_train)

print("R^2: " + str(r2(b, X_train[VARS], y_train)))
print("Adjusted R^2: " + str(adj_r2(b, X_train[VARS], y_train)))
print("RMSE: " + str(rmse(b, X_test[VARS], y_test)))
print("AIC: " + str(aic(b,X_test[VARS],y_test)))
print("BIC: " + str(bic(b,X_test[VARS],y_test)))

qqplot(b, X_train[VARS], y_train)

In [ ]:
X_train,X_test, y_train, y_test = train_test_split(.5, df_agg,"Volume Sold (Liters) per Capita")

VARS = ["Year","Quarter","Average Emp per Capita","Wages per Capita"] 
b = linear_regression(X_train[VARS], y_train)

print("R^2: " + str(r2(b, X_train[VARS], y_train)))
print("Adjusted R^2: " + str(adj_r2(b, X_train[VARS], y_train)))
print("RMSE: " + str(rmse(b, X_test[VARS], y_test)))
print("AIC: " + str(aic(b,X_test[VARS],y_test)))
print("BIC: " + str(bic(b,X_test[VARS],y_test)))

qqplot(b, X_train[VARS], y_train)